In [ ]:
import pandas as pd
import numpy as np

In [ ]:
def funcao_matriz(arquivos, data_inicio, data_fim):
    
    df = transformando_em_df(arquivos)

    df = separando_por_meses_selecionados(df, data_inicio, data_fim)

    df = agregar_subordinadas_senior(df)

    df = agregar_por_prazo(df)

    df = somar_ativos_em_uma_coluna(df)

    df = criando_coluna_provisao(df)

    df = manter_colunas_selecionadas(df)

    df, linhas_liquidadas = eliminando_linhas_liquidadas(df)

    df = substituindo_valores_nulos_de_algumas_colunas(df)

    df, linhas_com_nulos = eliminando_linhas_com_valores_nulos(df)

    df = label_encoder(df)

    df_predict = criando_dfPredict(df)

    df_predict = minmax_scaler(df_predict)

    y_pred = predictModel(df_predict)

    df = criando_coluna_com_predicao(df, y_pred)

    df = filtrando_linhas_em_alerta(df)

    df, linhas_liquidadas, linhas_com_nulos = selecionando_valores_relevantes_para_o_site(df, linhas_liquidadas, linhas_com_nulos)

    df_array, linhas_liquidadas_array, linhas_com_nulos_array = criando_arrays_para_cada_df(df, linhas_liquidadas, linhas_com_nulos)

    return df_array, linhas_liquidadas_array, linhas_com_nulos_array


In [ ]:
def transformando_em_df(arquivos):
    df_1 = pd.read_csv(arquivos[1])
    df_2 = pd.read_csv(arquivos[0])
    df = pd.merge(df_1, df_2, on='SK_Documento', how='inner')

    return df
    

In [ ]:
from datetime import datetime

def ajustando_data_do_dataframe(row):
    numeros = row.split('-')

    ano = numeros[0]

    mes = numeros[1]

    nova_data = f'{ano}-{mes}'

    return nova_data
    


def separando_por_meses_selecionados(df, data_inicial, data_final):
    
    data_inicial = datetime.strptime(data_inicial, "%Y-%m")
    data_final = datetime.strptime(data_final, "%Y-%m")

    datas_entre = []

    while data_inicial <= data_final:
        datas_entre.append(data_inicial.strftime("%Y-%m"))
        if data_inicial.month == 12:
            data_inicial = datetime(data_inicial.year + 1, 1, 1)
        else:
            data_inicial = datetime(data_inicial.year, data_inicial.month + 1, 1)



    df['Data_Competencia'] = df['Data_Competencia'].apply(ajustando_data_do_dataframe)


    df['Data_Competencia'] = pd.to_datetime(df['Data_Competencia'], format='%Y-%m')

    df = df[df['Data_Competencia'].dt.strftime('%Y-%m').isin(datas_entre)]

    

    return df
        

In [ ]:
def agregar_subordinadas_senior(df):
  df.loc[df['Classe_Serie'].str.contains("sub", case=False), 'Classe_Serie'] = "Subordinada"
  df.loc[df['Classe_Serie'].str.contains("nior", case=False), 'Classe_Serie'] = "Senior"

  classe_serie_types = ["Subordinada", "Senior"]
  cols_to_sum = ["Numero_Cotistas", "Quantidade_Cotas", "Valor_Cota", "Rentabilidade", "Valor_Total_Captado", "Quantidade_Cotas_Emitidas", "Valor_Total_Resgates", "Quantidade_Cotas_Resgatadas", "Valor_A_Pagar", "Quantidade_Cotas_A_Resgatar", "Valor_Amortizado_Cota", "Valor_Total_Amortizacao", "Desempenho_Esperado", "Desempenho_Realizado"]
  for classe_serie_type in classe_serie_types:
    for col in cols_to_sum:
      df[col + "_" + classe_serie_type] = df[df['Classe_Serie'] == classe_serie_type].groupby('ID_Documento')[col].transform('sum')

  df.drop(cols_to_sum, axis=1, inplace=True)
  df = df.drop_duplicates()

  df = df.groupby("ID_Documento", as_index=False).agg("first")

  return df




In [ ]:
def agregar_por_prazo(df):
    df['Carteira_Direitos_Aquisicao_Prazo_CP'] = df['Carteira_Direitos_Aquisicao_Prazo_1_30_dias'] + df['Carteira_Direitos_Aquisicao_Prazo_31_60_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_61_90_Dias']
    df['Carteira_Direitos_Aquisicao_Prazo_MP'] = df['Carteira_Direitos_Aquisicao_Prazo_91_120_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_121_150_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_151_180_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_181_360_Dias']
    df['Carteira_Direitos_Aquisicao_Prazo_LP'] = df['Carteira_Direitos_Aquisicao_Prazo_361_720_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias'] + df['Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias']
    df['Carteira_Direitos_Aquisicao_Inadimplentes_CP'] = df['Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias']
    df['Carteira_Direitos_Aquisicao_Inadimplentes_MP'] = df['Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias']
    df['Carteira_Direitos_Aquisicao_Inadimplentes_LP'] = df['Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias'] + df['Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias']
    df['Carteira_Direitos_Aquisicao_Pagos_CP'] = df['Carteira_Direitos_Aquisicao_Pagos_1_30_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_31_60_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_61_90_Dias']
    df['Carteira_Direitos_Aquisicao_Pagos_MP'] = df['Carteira_Direitos_Aquisicao_Pagos_91_120_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_121_150_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_151_180_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_181_360_Dias']
    df['Carteira_Direitos_Aquisicao_Pagos_LP'] = df['Carteira_Direitos_Aquisicao_Pagos_361_720_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias'] + df['Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Prazo_CP'] = df['Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Prazo_MP'] = df['Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Prazo_LP'] = df['Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_CP'] = df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_MP'] = df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_LP'] = df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Pagos_CP'] = df['Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Pagos_MP'] = df['Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias']
    df['Carteira_Direitos_Sem_Aquisicao_Pagos_LP'] = df['Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias'] + df['Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias']
    df['Liquidez_CP'] = df['Liquidez_Ate_30_Dias'] + df['Liquidez_Ate_60_Dias'] + df['Liquidez_Ate_90_Dias']
    df['Liquidez_MP'] = df['Liquidez_Ate_180_Dias'] + df['Liquidez_Ate_360_Dias']
    df['Liquidez_LP'] = df['Liquidez_Acima_360_Dias']
    
    agg_cols_to_drop = ['Carteira_Direitos_Aquisicao_Prazo_1_30_dias', 'Carteira_Direitos_Aquisicao_Prazo_31_60_Dias', 'Carteira_Direitos_Aquisicao_Prazo_61_90_Dias', 'Carteira_Direitos_Aquisicao_Prazo_91_120_Dias', 'Carteira_Direitos_Aquisicao_Prazo_121_150_Dias', 'Carteira_Direitos_Aquisicao_Prazo_151_180_Dias', 'Carteira_Direitos_Aquisicao_Prazo_181_360_Dias', 'Carteira_Direitos_Aquisicao_Prazo_361_720_Dias', 'Carteira_Direitos_Aquisicao_Prazo_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Prazo_Acima_1080_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_31_60_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_61_90_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_91_120_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_121_150_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_151_180_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_181_360_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_361_720_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Inadimplentes_Acima_1080_Dias', 'Carteira_Direitos_Aquisicao_Pagos_1_30_Dias', 'Carteira_Direitos_Aquisicao_Pagos_31_60_Dias', 'Carteira_Direitos_Aquisicao_Pagos_61_90_Dias', 'Carteira_Direitos_Aquisicao_Pagos_91_120_Dias', 'Carteira_Direitos_Aquisicao_Pagos_121_150_Dias', 'Carteira_Direitos_Aquisicao_Pagos_151_180_Dias', 'Carteira_Direitos_Aquisicao_Pagos_181_360_Dias', 'Carteira_Direitos_Aquisicao_Pagos_361_720_Dias', 'Carteira_Direitos_Aquisicao_Pagos_721_1080_Dias', 'Carteira_Direitos_Aquisicao_Pagos_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_1_30_dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Prazo_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_Acima_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_1_30_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_31_60_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_61_90_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_91_120_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_121_150_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_151_180_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_181_360_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_361_720_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_721_1080_Dias', 'Carteira_Direitos_Sem_Aquisicao_Pagos_Acima_1080_Dias', 'Liquidez_Ate_30_Dias', 'Liquidez_Ate_60_Dias', 'Liquidez_Ate_90_Dias', 'Liquidez_Ate_180_Dias', 'Liquidez_Ate_360_Dias', 'Liquidez_Acima_360_Dias']
    df.drop(agg_cols_to_drop, axis=1, inplace=True)

    return df

In [ ]:
def somar_ativos_em_uma_coluna(df):
    ativos_drop = ["Ativo_Valores_Mobiliarios", "Ativo_Titulos_Federais", "Ativo_CDB", "Ativo_Operacoes_Compromissadas", "Ativo_Outros_Ativos_Renda_Fixa", "Ativo_Cotas_FIDC", "Ativo_Warrants", "Ativo_Posicao_Derivativos", "Ativo_Mercado_Termo", "Ativo_Mercado_Opcoes", "Ativo_Mercado_Futuro", "Ativo_Diferencial_Swap", "Ativo_Depositos_Margem", "Ativo_Debentures", "Ativo_CRI", "Ativo_Notas_Promissorias", "Ativo_Letras Financeiras", "Ativo_Cotas_Fundos_ICVM_555", "Ativo_Outros_Valores_Mobiliarios", "Ativo_Cotas_FIDC_NP", "Ativo_Provisao_Debentures_CRI_NP_LF", "Ativo_Provisao_Cotas_FIDC", "Ativo_Provisao_Outros_Ativos"]

    for ativo in ativos_drop:
      df["Ativo_Outros"] = df["Ativo_Outros"] +  df[ativo]

    df = df.drop(ativos_drop, axis=1)

    return df

In [ ]:
def criando_coluna_provisao(df):
    df["Provisao"] = df["Ativo_Direitos_Aquisicao_Provisao_Reducao"] + df["Ativo_Direitos_Sem_Aquisicao_Provisao_Reducao"]

    return df

In [ ]:
def manter_colunas_selecionadas(df):
    columns_to_keep = ["Data_Competencia", "CNPJ", 'Nome_Fundo', "CNPJ_Administrador", "Forma_Condominio", "Patrimonio_Liquido", "Carteira_Direitos_Aquisicao_Inadimplentes_CP", "Carteira_Direitos_Aquisicao_Inadimplentes_MP", "Carteira_Direitos_Aquisicao_Inadimplentes_LP", "Carteira_Direitos_Sem_Aquisicao_Inadimplentes_CP", "Carteira_Direitos_Sem_Aquisicao_Inadimplentes_MP", "Carteira_Direitos_Sem_Aquisicao_Inadimplentes_LP", "Liquidez_CP", "Liquidez_MP", "Liquidez_LP", "Carteira_Financeiro", "Provisao", "Quantidade_Cotas_Senior", "Valor_Cota_Senior", "Rentabilidade_Senior", "Valor_Total_Captado_Senior", "Valor_Total_Resgates_Senior", "Valor_A_Pagar_Senior", "Quantidade_Cotas_A_Resgatar_Senior", "Valor_Amortizado_Cota_Senior", "Valor_Total_Amortizacao_Senior", "Desempenho_Esperado_Senior", "Desempenho_Realizado_Senior", "Quantidade_Cotas_Subordinada", "Valor_Cota_Subordinada", "Rentabilidade_Subordinada", "Valor_Total_Captado_Subordinada", "Valor_Total_Resgates_Subordinada", "Valor_A_Pagar_Subordinada", "Quantidade_Cotas_A_Resgatar_Subordinada", "Valor_Amortizado_Cota_Subordinada", "Valor_Total_Amortizacao_Subordinada", "Desempenho_Esperado_Subordinada", "Desempenho_Realizado_Subordinada"]
    df = df[columns_to_keep]

    return df

In [ ]:
def eliminando_linhas_liquidadas(df):
    colunas = df.columns.tolist()
    colunas_padrao = ['CNPJ', 'Nome_Fundo', 'Data_Competencia', 'CNPJ_Administrador', 'Forma_Condominio']
    for colunasP in colunas_padrao:
      colunas.remove(colunasP)
    
    linhas_liquidadas = df[df[colunas].eq(0).all(axis=1)]

    df = df.drop(linhas_liquidadas.index)

    return df, linhas_liquidadas

In [ ]:
def substituindo_valores_nulos_de_algumas_colunas(df):
    colunas = ['Carteira_Direitos_Aquisicao_Inadimplentes_CP', 'Carteira_Direitos_Aquisicao_Inadimplentes_MP', 'Carteira_Direitos_Aquisicao_Inadimplentes_LP', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_CP', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_MP', 'Carteira_Direitos_Sem_Aquisicao_Inadimplentes_LP', 'Liquidez_CP', 'Liquidez_MP', 'Liquidez_LP']
    for coluna in colunas:
      df[coluna] = df[coluna].fillna(0)

    return df

In [ ]:
def eliminando_linhas_com_valores_nulos(df):
    linhas_com_nulos = df[df.isnull().any(axis=1)]
    df = df.dropna()
    
    return df, linhas_com_nulos

In [ ]:
from sklearn.preprocessing import LabelEncoder

def label_encoder(df):
    var_mod =['Forma_Condominio']
    le = LabelEncoder()
    for i in var_mod:
      df[i] = le.fit_transform(df[i])

    return df

In [ ]:
def criando_dfPredict(df):

    df_predict = df.drop(['Nome_Fundo', 'CNPJ', 'CNPJ_Administrador', 'Data_Competencia'], axis=1)

    return df_predict

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def minmax_scaler(df):
    scaler = MinMaxScaler()
    df = scaler.fit_transform(df)

    return df

In [ ]:
import treelite

def predictModel(df):
    checkpoint_path = './modeloPreditivo/RandomForestClassifier.tl'
    tl_model = treelite.Model.deserialize(checkpoint_path)
    y_pred = treelite.gtil.predict(tl_model, df, pred_margin=True)

    return y_pred
    

In [ ]:
def criando_coluna_com_predicao(df, y_pred):
    df['y_pred'] = y_pred[:, 1]

    return df

In [ ]:
def filtrando_linhas_em_alerta(df):
    filtro = df['y_pred'] >= 0.5

    df = df[filtro]

    return df

In [ ]:
def selecionando_valores_relevantes_para_o_site(df1, df2, df3):
    colunas_para_o_site = ['Nome_Fundo', 'CNPJ']

    df1 = df1[colunas_para_o_site]
    df2 = df2[colunas_para_o_site]
    df3 = df3[colunas_para_o_site]

    return df1, df2, df3

In [ ]:
def criando_arrays_para_cada_df(df1, df2, df3):
    df1_array = []

    for index, row in df1.iterrows():
        array_da_linha = row.tolist()
        df1_array.append(array_da_linha)


    df2_array = []

    for index, row in df2.iterrows():
        array_da_linha = row.tolist()
        df2_array.append(array_da_linha)
        

    df3_array = []

    for index, row in df3.iterrows():
        array_da_linha = row.tolist()
        df3_array.append(array_da_linha)


    return df1_array, df2_array, df3_array

    

In [ ]:
from flask import Flask, render_template, request,  jsonify
app = Flask(__name__, template_folder='templates')
@app.route("/", methods = ['Get','POST'])
def home_get():
    if request.method == 'GET':
        return render_template('index.html')
    if request.method == 'POST':
        arquivos = request.files.getlist("arquivos")
        mes_inicial = request.form.get("mes_inicial")
        mes_final = request.form.get("mes_final")

        principal_array, liquidadas_array, nulos_array = funcao_matriz(arquivos, mes_inicial, mes_final)

        return render_template("index_predict.html", principal_array = principal_array, liquidadas_array = liquidadas_array, nulos_array = nulos_array)

@app.route("/quemSomos")
def quemSomos():
    return render_template('quemSomos.html')

@app.route("/contato")
def contato():
    return render_template('contato.html')



if __name__ == "__main__":
    app.run()